In [1]:
import duckdb
import pyarrow as pa
import pyarrow.dataset as ds
import pyarrow.parquet as pq
from tqdm import tqdm

In [5]:
import os

os.getcwd()

'/home/flavio/repositories/lstk-dataset/notebooks'

In [11]:
authors_ds = ds.dataset("../data/authors.parquet")
# authors_db = duckdb.arrow(authors_ds)

In [12]:
affiliations_ds = ds.dataset("../data/affiliations.parquet")


In [15]:
with duckdb.connect() as con:
    chunk_size = 50_000
    query = con.execute("""
        WITH ranked_duplicates AS (
            SELECT 
                *,
                ROW_NUMBER() OVER (
                    PARTITION BY AuthorId, Year
                    ORDER BY random()
                ) as row_num
            FROM affiliations_ds
        )
        SELECT * FROM authors_ds
        LEFT JOIN (
            SELECT affiliations.AuthorId, 
            Iso3166Code as Iso3166Code_FirstAffiliation
            FROM ranked_duplicates AS affiliations 
            INNER JOIN (
              SELECT AuthorId, 
              MIN(Year) AS first_affiliation_year
              FROM ranked_duplicates 
              GROUP BY AuthorId
            ) as b 
            ON (affiliations.authorid=b.authorid 
                and affiliations.year=b.first_affiliation_year
            )
            WHERE row_num = 1
        )
        USING (AuthorId) 
    """)
    record_batch_reader = query.fetch_record_batch(chunk_size)
    first_chunk = next(record_batch_reader)
    with pq.ParquetWriter("../data/mytest.parquet", first_chunk.schema) as writer:
        writer.write(first_chunk)
        for i, chunk in tqdm(enumerate(record_batch_reader), desc="Processing batches"):
            writer.write(chunk)








Processing batches: 21it [00:00, 32.34it/s]


## We also need to get rid of duplicates in affiliation table

In [ ]:
affiliations_ds = ds.dataset("../data/affiliations.parquet")
